In [ ]:
import numpy as np
import graphinglib as gl
import pyregion
from scipy.optimize import curve_fit
import scipy
import dill
from functools import partial

from src.tools.statistics.advanced_stats import autocorrelation_function, autocorrelation_function_2d
from src.hdu.maps.map import Map

In [ ]:
def get_plottables(data: np.ndarray, region_radius: float) -> list:
    sorted_data = data[np.argsort(data[:,0])]
    scatter = gl.Scatter(
        sorted_data[:,0],
        sorted_data[:,1],
        marker_size=2,
        face_color="orange",
    )

    scatter.add_errorbars(
        y_error=sorted_data[:,2],
        errorbars_line_width=0.25,
        cap_width=0,
    )

    return [scatter]

In [ ]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("Global region", None, 50),
    ("Diffuse region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 20),
    ("Central region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 10),
    ("Filament region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 6)
]

### Autocorrelation functions of all regions

In [ ]:
figs = []

for name, region, region_radius in regions:
    # if not region:
    #     try:
    #         with open("applications/sh158/saved_data/acr_func_turbulence_global.gz", "rb") as f:
    #             data = dill.load(f)
    #     except:
    #         data = autocorrelation_function(turbulence.get_masked_region(region).data)
    #         with open("applications/sh158/saved_data/acr_func_turbulence_global.gz", "wb") as f:
    #             dill.dump(data, f)
    # else:
    #     data = autocorrelation_function(turbulence.get_masked_region(region).data)
    test = Map.load("summer_2023/gaussian_fitting/maps/external_maps/"
                      +"dens_it_sii_sans_fcorr_nii_plus_plusmin_pouss_seuil_errt_1000.fits")

    # data = autocorrelation_function(test.data)
    data = autocorrelation_function(turbulence.get_masked_region(region).data)

    fig = gl.Figure(
        title=name,
        x_label="Lag",
        y_label="Autocorrelation function",
        x_lim=(0, region_radius*1.2),
    )
    plottables = get_plottables(data, region_radius)
    cropped_scatter = plottables[0].create_slice_x(0, region_radius*1.2)
    fig.y_lim = np.min(cropped_scatter.y_data)-0.2, np.max(cropped_scatter.y_data)+0.3
    fig.add_elements(*plottables)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
multifig.save("figures/sh158/advanced_stats/autocorrelation/autocorrelation_functions_new.pdf")

### 2D Autocorrelation function

In [ ]:
data = autocorrelation_function_2d(turbulence.data)

# Copy paste the data with a diagonal reflection
data = np.append(data, data * np.tile((-1, -1, 1), (data.shape[0], 1)), axis=0)

# data[data[:,2] > 1,2] = np.NAN
# data[data[:,2] < -1,2] = np.NAN

fig = gl.Figure(
    x_lim=(np.min(data[:,0]), np.max(data[:,0])),
    y_lim=(np.min(data[:,1]), np.max(data[:,1])),
)

x_grid, y_grid = np.meshgrid(np.arange(fig.x_lim[0], fig.x_lim[1] + 1), 
                             np.arange(fig.y_lim[0], fig.y_lim[1] + 1))

z_data = np.zeros_like(x_grid)
for x, y, z in data:
    z_data[int(y-np.min(data[:,1])), int(x-np.min(data[:,0]))] = z
z_data = scipy.ndimage.gaussian_filter(z_data, 3)

for dat in data:
    if dat[0] == 0 and dat[1] == 0:
        zero_lag_val =  dat[2]
        break
else:
    raise Exception("No zero lag")

scat = gl.Scatter(
    x_data=data[:,0],
    y_data=data[:,1],
    face_color=data[:,2] / zero_lag_val,
    show_color_bar=True,
    marker_size=2,
)

cont = gl.Contour(
    x_mesh=x_grid,
    y_mesh=y_grid,
    z_data=z_data / zero_lag_val,
    show_color_bar=True,
    number_of_levels=list(np.arange(-1, 1 + 0.1, 0.1)),
    filled=False,
    color_map="viridis",
)
fig.add_elements(cont)
fig.save("figures/sh158/advanced_stats/autocorrelation/autocorrelation_function_2d_mirror_cont_new.pdf")
# fig.add_elements(scat)
# fig.save("figures/sh158/advanced_stats/autocorrelation/autocorrelation_function_2d_mirror_cont_scat.pdf")

# Tests

In [ ]:
arr = np.array(np.random.random((1000,1000)))
arr[0,0] = np.nan
print(arr)
print(np.nanmean(arr))
print(autocorrelation_function_2d(arr))
print(arr)